In [2]:
import os
import sys
import subprocess
import shutil
import pandas as pd
import numpy as np
import xmltodict
import json
import hashlib
import requests
import getpass
from requests import Response

import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import table

HOMEDIR="."

In [4]:
sys.path.append("../../lib")
print(sys.path)
from pyutils import *
from pdutils import *
from pdpltutils import *
from gputils import *
from iputils import *

['/home/jblake1/git/LELPerformance/Network_Measurement_Project/Jupyter', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/jblake1/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '../../lib']


In [5]:
DIGIIP="192.168.2.1"
DIGIPW="Cmu5@ty@l@60"
DIGI_TESTCMD="config network modem wwan1 sim_slot"

In [14]:
basecmd = f"sshpass -p {DIGIPW} ssh admin@{DIGIIP}"
cmd=f"{basecmd} {DIGI_TESTCMD}"

In [7]:
print(cmd)

sshpass -p Cmu5@ty@l@60 ssh admin@192.168.2.1 config network modem wwan1 sim_slot


In [9]:
oscmd(cmd)

0

In [262]:
def runDIGIcmd(icmd, output = True):
    cmd=f"{basecmd} {icmd}"
    return cmd_all(cmd, output=output)
    
runDIGIcmd(DIGI_TESTCMD + " 2",output=False)
runDIGIcmd(DIGI_TESTCMD, output=False)

{'stdout': ['2', ''], 'stderr': []}

In [261]:
def getModemStatus(output=True):
    res = runDIGIcmd("show modem verbose",output=output)
    modemstat = None
    stdoutclean = [line.strip() for line in res['stdout'] if line.startswith(" Modem") or line.startswith(" wwan1")]
    if len(stdoutclean) < 2:
        print("Modem is not connected")
        modemstat = 'not_connected'
    elif 'disabled' in stdoutclean[1]:
        print("Modem is disabled")
        modemstat = 'disabled'
    elif 'found' in stdoutclean[1]:
        print("Modem is not found")
        modemstat = 'not_found'
    elif 'enabling' in stdoutclean[1]:
        print("Modem is being enabled")
        modemstat = 'enabling'
    else:
        data = [fld for fld in stdoutclean[1].split(' ') if fld != '']
        dbm = data[-2].replace("(","")
        modemstat = f"{data[3]}_SIM:{data[1]}_DBM:{dbm}"
        if output == True: print(data)
    return modemstat,res


modemstat, output = getModemStatus(output=False)
print(modemstat)

connected_SIM:1_DBM:-75


In [236]:
getModemStatus(output=False)

['wwan1', '2', '(ready)', 'connected', 'Excellent', '(-40', 'dBm)']


('connected_SIM:2_DBM:-40',
 ['wwan1', '2', '(ready)', 'connected', 'Excellent', '(-40', 'dBm)'])

In [263]:
TIMEOUTCT=40
TIMEOUTWT=5

def waitForConnect():
    for ii in range(0,TIMEOUTCT):
        mstat, output = getModemStatus(output=False)
        nonstatelst = ['not_connected','not_found','disabled','enabling']
        if (mstat not in nonstatelst):
            print(mstat)
            break
        else:
            print(output)
        print(f"{ii}")
        sleep(TIMEOUTWT)

waitForConnect()

Modem is disabled
{'stdout': ['', ' Modem  SIM        Status    APN  Signal Strength', ' -----  ---------  --------  ---  -------------------', ' wwan1  2 (ready)  disabled       Excellent (-41 dBm)', '', ''], 'stderr': []}
0
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
1
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
2
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
3
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
4
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
5
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
6
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
7
Modem is not connected
{'stdout': ['', ' Modem not available.', '', ''], 'stderr': []}
8
Modem is not connected
{'stdout': ['', ' Modem not available.'

In [163]:
_ = getModemStatus(output=False)

Modem is not connected


In [167]:
_ = getModemStatus(output=False)

['wwan1', '2', '(ready)', 'connected', 'Excellent', '(-40', 'dBm)']


## Remote Manager Web API

In [73]:
#!python3
# Python example using requests library 
# Use 'pip install requests' to install the requests library.           

host="remotemanager.digi.com"            
api="https://%s/ws/v1/devices/inventory/00000000-00000000-0004F3FF-FF94F583/" % host

# user = input("Username for %s: " % host)
user="jblake1@andrew.cmu.edu"
# password = getpass.getpass("Password for user %s at %s:" % (user, host))
password="Cmu5@ty@l@60"

# Retaining a session when doing multiple requests can speed up your requests
session = requests.Session()
session.auth = (user, password)

rsp = session.get(api)
rsp.raise_for_status()
print("Status code:", rsp.status_code)
# print("Result text:", rsp.text)

stts=json.loads(rsp.text)

for xx in stts.keys():
    print(f"{xx} -> {stts[xx]}")

Status code: 200
Result text: {"channels_uri":"/ws/v1/devices/channels/00000000-00000000-0004F3FF-FF94F583","metrics_uri":"/ws/v1/devices/metrics/00000000-00000000-0004F3FF-FF94F583","capabilities":{"sm_compression_available":true,"sm_pack_available":true,"sms_enabled":true,"cli_service_available":true,"sm_encryption_key_distribution_service_available":true,"speedtest_available":true,"modem_firmware_update_available":true,"modem_firmware_advanced_available":true,"speedtest_advanced_available":true},"customer_id":101399,"type":"Digi IX40","name":"CMU-Digi-IX40-01","vendor_id":4261412874,"public_ip":"172.59.138.238","ip":"192.0.0.2","connection_status":"connected","id":"00000000-00000000-0004F3FF-FF94F583","restricted_status":"unrestricted","serial_number":"IX40182103362438","sku":"IX40-05","cellular_modem_id":"359918921552923","iccid":"8901260318739898049","maintenance_mode":"off","firmware_version":"24.6.17.54","firmware_status":"non_production","last_connect":"2024-11-15T15:55:38.830Z

In [97]:
from time import sleep
KEYLST = ['connection_status','provider','apn','carrier','last_connect','last_disconnect']
def checkStatus(output=False,keylst=KEYLST):
    session = requests.Session()
    session.auth = (user, password)
    rsp = session.get(api)
    rsp.raise_for_status()
#     print(rsp.text)
    stts=json.loads(rsp.text)
    if (output):
        keyresult = " ".join([stts[key] for key in keylst if key in stts ])
        print(keyresult)
    del rsp
    del session
    return stts

TIMEOUTCT=20
TIMEOUTWT=2

def waitForConnect():
    for ii in range(0,TIMEOUTCT):
        stts = checkStatus()
        connectstat=stts['connection_status']
        carrier = stts['provider']
        if (connectstat=='connected'):
            print(f"Connected to {carrier} last_disconnect={stts['last_disconnect']}")
#             break
        print(f"{ii}")
        sleep(TIMEOUTWT)
                  
# waitForConnect()

In [99]:
checkStatus(output=True)

connected T-Mobile fast.t-mobile.com LivingEdgeLab 2024-11-15T16:31:59.887Z 2024-11-15T16:09:17.690Z


{'channels_uri': '/ws/v1/devices/channels/00000000-00000000-0004F3FF-FF94F583',
 'metrics_uri': '/ws/v1/devices/metrics/00000000-00000000-0004F3FF-FF94F583',
 'capabilities': {'sm_compression_available': True,
  'sm_pack_available': True,
  'sms_enabled': True,
  'cli_service_available': True,
  'sm_encryption_key_distribution_service_available': True,
  'speedtest_available': True,
  'modem_firmware_update_available': True,
  'modem_firmware_advanced_available': True,
  'speedtest_advanced_available': True},
 'firmware_version': '24.6.17.54',
 'firmware_status': 'non_production',
 'last_connect': '2024-11-15T16:31:59.887Z',
 'last_disconnect': '2024-11-15T16:09:17.690Z',
 'customer_id': 101399,
 'type': 'Digi IX40',
 'name': 'CMU-Digi-IX40-01',
 'vendor_id': 4261412874,
 'public_ip': '128.2.209.111',
 'ip': '192.168.128.249',
 'connection_status': 'connected',
 'id': '00000000-00000000-0004F3FF-FF94F583',
 'restricted_status': 'unrestricted',
 'serial_number': 'IX40182103362438',
 'sk

## Local Web API

In [117]:
#!python3
# Python example using requests library 
# Use 'pip install requests' to install the requests library.           

host="192.168.2.1"            
api="https://%s/cgi-bin/config.cgi/value/service/ssh" % host
print(api)
# https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh

# user = input("Username for %s: " % host)
user="admin"
# password = getpass.getpass("Password for user %s at %s:" % (user, host))
password="Cmu5@ty@l@60"

# # Retaining a session when doing multiple requests can speed up your requests
# session = requests.Session()
# session.auth = (user, password)

# rsp = session.get(api)
# rsp.raise_for_status()
# print("Status code:", rsp.status_code)
# # print("Result text:", rsp.text)

cmd=f"curl -k -u {user}:{password} {api}"
print(cmd)
response = cmd_all(cmd, output=True)

# stts=json.loads(rsp.text)

# for xx in stts.keys():
#     print(f"{xx} -> {stts[xx]}")

https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh
curl -k -u admin:Cmu5@ty@l@60 https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh
STDOUT
{
	"ok": true,
	"result": {
		"type": "object",
		"path": "service.ssh"
,		"collapsed": {
"acl.zone.0": "internal"
, 
"acl.zone.1": "edge"
, 
"acl.zone.2": "ipsec"
, 
"acl.zone.3": "setup"
, 
"custom.config_file": ""
, 
"custom.enable": "false"
, 
"custom.override": "false"
, 
"enable": "true"
, 
"key": ""
, 
"mdns.enable": "true"
, 
"mdns.name": ""
, 
"mdns.type": "_ssh._tcp."
, 
"port": "22"
, 
"protocol.0": "tcp"
		}
	}
}

STDERR
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   429    0   429    0     0   1941      0 --:--:-- --:--:-- --:--:--  1941


In [121]:
# config network modem wwan1 sim_slot
api="https://%s/cgi-bin/config.cgi/value/network/interface/wwan1" % host
cmd=f"curl -k -u {user}:{password} {api} -X GET"
print(cmd)
response = cmd_all(cmd, output=True)

curl -k -u admin:Cmu5@ty@l@60 https://192.168.2.1/cgi-bin/config.cgi/value/network/interface/wwan1 -X GET
STDOUT
{
	"ok": true,
	"result": {
		"type": "object",
		"path": "network.interface.wwan1"
,		"collapsed": {
"enable": "true"
, 
"ipv4.enable": "true"
, 
"ipv4.metric": "3"
, 
"ipv4.mgmt": "0"
, 
"ipv4.modem_type": "static"
, 
"ipv4.mtu": "1500"
, 
"ipv4.use_dns": "primary"
, 
"ipv4.weight": "10"
, 
"ipv6.enable": "true"
, 
"ipv6.metric": "3"
, 
"ipv6.mgmt": "0"
, 
"ipv6.modem_type": "static"
, 
"ipv6.mtu": "1500"
, 
"ipv6.use_dns": "primary"
, 
"ipv6.weight": "10"
, 
"modem.apn.0.apn": ""
, 
"modem.apn.0.attm2mglobal": "true"
, 
"modem.apn.0.auth": "none"
, 
"modem.apn.0.cid": "0"
, 
"modem.apn.0.ip_version": "auto"
, 
"modem.apn.0.password": ""
, 
"modem.apn.0.username": ""
, 
"modem.apn_selection": "built_in_list_only"
, 
"modem.backoff_enabled": "false"
, 
"modem.carrier": "AT&T"
, 
"modem.carrier_other": ""
, 
"modem.connection_timeout": "30"
, 
"modem.custom_gw.enable": "fals

In [109]:
cmd="curl -k -u admin:Cmu5@ty@l@60 https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh"
cmd_all(cmd,output=True)

STDOUT
{
	"ok": true,
	"result": {
		"type": "object",
		"path": "service.ssh"
,		"collapsed": {
"acl.zone.0": "internal"
, 
"acl.zone.1": "edge"
, 
"acl.zone.2": "ipsec"
, 
"acl.zone.3": "setup"
, 
"custom.config_file": ""
, 
"custom.enable": "false"
, 
"custom.override": "false"
, 
"enable": "true"
, 
"key": ""
, 
"mdns.enable": "true"
, 
"mdns.name": ""
, 
"mdns.type": "_ssh._tcp."
, 
"port": "22"
, 
"protocol.0": "tcp"
		}
	}
}

STDERR
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   429    0   429    0     0   2004      0 --:--:-- --:--:-- --:--:--  2014


{'stdout': ['{',
  '\t"ok": true,',
  '\t"result": {',
  '\t\t"type": "object",',
  '\t\t"path": "service.ssh"',
  ',\t\t"collapsed": {',
  '"acl.zone.0": "internal"',
  ', ',
  '"acl.zone.1": "edge"',
  ', ',
  '"acl.zone.2": "ipsec"',
  ', ',
  '"acl.zone.3": "setup"',
  ', ',
  '"custom.config_file": ""',
  ', ',
  '"custom.enable": "false"',
  ', ',
  '"custom.override": "false"',
  ', ',
  '"enable": "true"',
  ', ',
  '"key": ""',
  ', ',
  '"mdns.enable": "true"',
  ', ',
  '"mdns.name": ""',
  ', ',
  '"mdns.type": "_ssh._tcp."',
  ', ',
  '"port": "22"',
  ', ',
  '"protocol.0": "tcp"',
  '\t\t}',
  '\t}',
  '}',
  ''],
 'stderr': ['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
  '                                 Dload  Upload   Total   Spent    Left  Speed',
  '\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100   429    0   429    0     0   2004      0 --:--:-- --:--:-- --:--:--  2014']}